# LTL-Generation, Filtering, Translation to TGBA

In [17]:
!genltl --dac --eh --hkrss --p --sb | ltlfilt --relabel=abc > literature.ltl
!ltl2tgba --tgba -F 'literature.ltl' | autfilt -v --is-empty | autfilt --acc-sets=2.. --unique > literature.hoa

# Evaluation

In [25]:
!autcross --timeout=60 -F 'literature.hoa' -t\
'{SaG+}python3.9 ../SimpleAndGeneral/complement.py --skip --jump_levels %H | autfilt --small -b >%O'\
'{Autfilt+}autfilt --complement --small -b'\
'{Seminator+}tools/seminator/seminator --complement %H | autfilt --small -b >%O'\
'{Kofola+}tools/kofola/kofola --algo=comp %H| autfilt --small -b >%O'\
--csv > Literature_Results.csv

literature.hoa:1.1-69.7	G(!a | G!b | ((!c | X((!b U (d & Fe)) | (b R !d))) U b))
Running [A0]: python3.9 ../SimpleAndGeneral/complement.py --skip --jump_levels 'lcr-i0-3U4cQ6' | autfilt --small -b >'lcr-o0-1tTUm4'
Running [A1]: autfilt --complement --small -b 'lcr-i0-Wji0f6'>'lcr-o1-MGIVM6'
Running [A2]: tools/seminator/seminator --complement 'lcr-i0-hYgt75' | autfilt --small -b >'lcr-o2-xPH7V5'
Running [A3]: tools/kofola/kofola --algo=comp 'lcr-i0-tqt3K4'| autfilt --small -b >'lcr-o3-YmKJQ4'
Performing sanity checks and gathering statistics...

literature.hoa:70.1-146.7	G(!a | ((!b | X((!c U (d & Fe)) | (c R !d))) W c))
Running [A0]: python3.9 ../SimpleAndGeneral/complement.py --skip --jump_levels 'lcr-i1-nrZdv3' | autfilt --small -b >'lcr-o0-d12XO4'
Running [A1]: autfilt --complement --small -b 'lcr-i1-4nG3V6'>'lcr-o1-XHMjI3'
Running [A2]: tools/seminator/seminator --complement 'lcr-i1-r7X8m3' | autfilt --small -b >'lcr-o2-0q1lj7'
Running [A3]: tools/kofola/kofola --algo=comp 'lcr-i1

# Data Transformation

In [28]:
import pandas as pd

data = pd.read_csv("Literature_Results.csv")

tools = ["SaG+", "Autfilt+", "Seminator+", "Kofola+"]

input_columns = [col for col in data.columns if col.startswith("input")]

results = []

for tool in tools:
    tool_results = data[data.tool == tool]
    tool_results = tool_results.rename(columns={"output.states" : tool})
    results.append(tool_results)
    
data = data[input_columns].drop_duplicates().rename(columns={"input.acc_sets" : "Acceptance Sets"})

for tool_results, tool in zip(results, tools):
    data = data.merge(tool_results[["input.source", tool]], on="input.source", how="inner")

data = data.drop_duplicates()

for tool in tools:
    data[tool] = pd.to_numeric(data[tool])

data.to_csv("Literature_Benchmark.csv")
